#Requirements

In [1]:
import pandas as pd
import numpy as np

In [3]:
%%capture
!pip install huggingface_hub
!pip install datasets transformers
!apt install git-lfs

In [4]:
from huggingface_hub import notebook_login

notebook_login()

# Fine-tuning a language model

## Preparing the dataset

In [5]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
data = pd.read_csv('../data/ads.csv')

In [3]:
data = data.drop_duplicates()

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
data_train, data_valid = train_test_split(data, test_size=0.2, random_state=42)

In [8]:
data_train.to_csv('../data/train.csv', index = False)
data_valid.to_csv('../data/valid.csv', index = False)

In [11]:
%%capture
from datasets import load_dataset
dataset = load_dataset("csv", data_files={"train": '../data/train.csv', "validation": '../data/valid.csv'})

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [12]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset)
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [13]:
show_random_elements(dataset["train"])

,ads
0,أورنج، ريزو لي فين ماكنتي يجري بيك للقدام الريزو أورنج، جربتوه و للمغاربة ضمنتوه
1,للربح كل نهااار مع أورنج شارجي و حاول تربح على
2,ديما الجديد عند أورنج تكونكطى بآخر ماكين فالروتور مع و استافد من والويفي ، تغطية فالدار كاملة بالإضافة لتطبيق باش تهنا على وليداتك
3,الجدييييد مع فورفيات ، لمااااكس ديال السخاوة بأقل ثمن تبرع ب أنتيرنيت، ساعة ديال المكالمات، بالإضافة إلى اللامحدود على مواقع التواصل الإجتماعي و ال غير ب درهم
4,أورنج ترافق المقاولات الصغرى، المهنين و الحرفيين المغاربة زهرة رئيسة تعاونية و مستفيدة من قافلة ، تخبرنا عن ورشات العمل و تجربتها مع أورنج فالمجال الرقمي
5,وحدة من أسرار النجاح فهاد الوقت هو تكون مكونيكطي مع العالم شارجي تبع أشنو واقع و برز موهبتك حتى نتا
6,استفيدوا من مزايا الفيبر للاتصالات المغرب الاشتراك الفوري في الفيبر عبر أو الرقم الخاص التركيب في خلآل ساعة
7,بمناسبة كأس إفريقيا للأمم النسوية أورنج نظمت أول بطولة إفريقية لكرة القدم تحت شعار ، شارك فيها فرق، مشارك لي عشق الكرة كيجمعهم و هدفهم تمثيل بلدهم أحسن تمثيل أهداف ، دريبلات ، وبززاااف ديال المتعة عشتوها معانا شاهدو الفيديو باش تكتاشفو معانا شكون الفريق الفائز بالمسابقة
8,بارك لصحابك و ديالك نتا لول السنة الجديدة مع درهم فيها شهر كااامل على مواقع التواصل الإجتماعي
9,هنيئاً للفائزين لي شارجاو و ربحو معانا فمسابقة نمرة و ربحة ، بالإضافة لفائز جهة كلميم شارجيو حتى نتوما وجربو حظكم لربح موطور كل نهار مع أورنج، المسابقة مستمرة إلى غاية غشت


## Causal Language modeling

In [14]:
model_checkpoint = "aubmindlab/aragpt2-base"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [16]:
def tokenize_function(examples):
    return tokenizer(examples["ads"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [17]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["ads"])

Map (num_proc=4):   0%|          | 0/209 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/53 [00:00<?, ? examples/s]

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [18]:
tokenized_dataset["train"][1]

{'input_ids': [526,
  1678,
  308,
  34041,
  281,
  5578,
  152,
  238,
  1011,
  49311,
  305,
  385,
  390,
  1471,
  308,
  21418,
  4739,
  262,
  33595,
  2637,
  430,
  1612,
  16963,
  355,
  4439,
  297,
  281,
  4165,
  41834,
  664,
  1890,
  2253,
  1022,
  308,
  11918,
  288,
  3676,
  412,
  62029,
  25443,
  152,
  238,
  6766,
  716,
  10450,
  760,
  581,
  262,
  25559,
  355,
  43380,
  20813,
  1982,
  15069,
  2387,
  281,
  38914,
  35536,
  308,
  1792,
  6645,
  8601],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [19]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [20]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.


In [21]:
lm_datasets = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/209 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/53 [00:00<?, ? examples/s]

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [22]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'، عرض صالح إلى غاية غشتعبؤوا رصيدكم عبر الانترنيت و شاركو تلقائياً ني في القرعة للفوز بالعديد من الهواتف الذكيةأورنج تدعوكم لورشات تكوينية حول التطور و التسويق الرقمي التسجيل علىاتبعوا معنا تجربة المقاول هشام بلعزري صاحب مشروع ، وقدر يزيد به القدام عن طريق الدورات التكوينية عبر برنامج بشراكة مع اورنجحيت عندنا ديما نتا لول نتا لول لي تعيط و تسول فالحباب و الصحاب مع نجمة ساعة من مكالمات غير ب دراهم صالحة سيييييمانةجدييييد عند أورنج بقى ديما كونيكطي إبتداءً من درهم فقط اللامحدود على مواقع التواصل الاجتماعي حتى'

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

In [23]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

And some `TrainingArguments`:

In [24]:
from transformers import Trainer, TrainingArguments

In [25]:
%%capture
!pip install accelerate -U
!pip install transformers[torch]

In [26]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}",
    #evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    #push_to_hub=True,
)

We pass along all of those to the `Trainer` class:

In [27]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

And we can train our model:

In [28]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=18, training_loss=9.185758802625868, metrics={'train_runtime': 9.2084, 'train_samples_per_second': 14.335, 'train_steps_per_second': 1.955, 'total_flos': 8622637056000.0, 'train_loss': 9.185758802625868, 'epoch': 3.0})

## Pushing the model, the tokenizer to the HuggingFace Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [35]:
tokenizer.push_to_hub("IssamL/darijabertgenad", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:844: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/IssamL/darijabertgenad/commit/30298e7d17bf9779b53c5d3ad71208f3fcf66a43', commit_message='Upload tokenizer', commit_description='', oid='30298e7d17bf9779b53c5d3ad71208f3fcf66a43', pr_url=None, pr_revision=None, pr_num=None)

In [34]:
model.push_to_hub("IssamL/darijabertgenad", use_auth_token=True)

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/IssamL/darijabertgenad/commit/4be8c6f615629ca2d5416457cc20f4d5fa543b71', commit_message='Upload model', commit_description='', oid='4be8c6f615629ca2d5416457cc20f4d5fa543b71', pr_url=None, pr_revision=None, pr_num=None)

#Inference

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "IssamL/darijabertgenad"
model = AutoModelForCausalLM.from_pretrained(model_name)
#tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Set the generation parameters
prompt = "مع عرض انوي"
max_length = 1000
temperature = 1.0
top_k = 0
top_p = 0.9
num_return_sequences = 1

# Generate text from the model
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=max_length,
    temperature=temperature,
    top_k=top_k,
    top_p=top_p,
    do_sample=True,
    num_return_sequences=num_return_sequences,
)
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


مع عرض انوي في الفترة القادمة من سيرك دبي الذي يقام مابين 8 الى 10 نوفمبر بتوقيت دبي امام هذا الجمهور المحترمين فهل تعلم اخي القارئ قيمة هذا المعرض وهل يكسب سمعة وفي الوقت الذي سيخبرك هذا الجمهور بهذا الشيء تاركين خطوة اخرى كانت مصيرية لتتكون جوائز تقديرا لهذه الشخصيات المرنة و على راسها السيدات بذالك انصح بالمشاركة في هذا المعارض المعروفة و الثلاثهم مجد رجل على الواقف و كلو دور جديد اكثر من فعالية اتمنى ان يعطي رايكم لهذه الفئة باحترام افضلقالت احدى صديقاته بالامس الاثنين الماضي ل نها هي من المنطقة لكن في الطريق خشيت على مرورها هذا اني غائبة عن السير
